In [79]:
import numpy as np
import random
import time

In [90]:
class Board:
    def __init__(self, n, heuristic_function=1) -> None:
        """
        heuristics_function: 1 or 2
        """
        self.n = n
        self.state = None
        self.initialize_board()
        self.seen_states = []
        self.heuristic_function = heuristic_function
        if heuristic_function == 1:
            self.heuristic = self.heuristic1
            self.goal = 0
        elif heuristic_function == 2:
            self.heuristic = self.heuristic2
            self.goal = self.n

        self.h_score = self.heuristic()

    def initialize_board(self):
        """
        Initialize the board with randm positions of queens.
        One queen per row.
        """
        self.state = np.zeros((self.n, self.n), dtype=np.int8)
        columns = list(range(self.n))
        random.shuffle(columns, random.random)  # Shuffle the columns in-place

        # Place the queens randomly
        for row, col in enumerate(columns):
            self.state[row, col] = 1

    def heuristic1(self, board=None):
        """
        Compute the heuristic value of a given state.
        The heuristic value is the number of pairs of queens that can attack each other.
        """
        if board is None:  # If no board is passed, use the current state
            board = self

        score = 0
        for row in range(board.n):
            for col in range(board.n):
                if board.state[row, col] == 1:  # If there is a queen in this position
                    # Check the column for other queens
                    score += np.sum(board.state[:, col]) - 1
                    # Check the diagonals for other queens
                    for i in range(1, 8):
                        if row + i < 8 and col + i < 8:
                            score += board.state[row + i, col + i]
                        if row - i >= 0 and col - i >= 0:
                            score += board.state[row - i, col - i]
                        if row + i < 8 and col - i >= 0:
                            score += board.state[row + i, col - i]
                        if row - i >= 0 and col + i < 8:
                            score += board.state[row - i, col + i]

        self.h_score = score
        return score

    def heuristic2(self, board=None):
        """
        Compute the heuristic value of a given state.
        The heuristic value is the number of queens that are not under attack.
        """
        if board is None:  # If no board is passed, use the current state
            board = self

        score = 0
        for row in range(board.n):
            for col in range(board.n):
                if board.state[row, col] == 1:  # If there is a queen in this position
                    temp_score = 0
                    # Check the column for other queens
                    temp_score += np.sum(board.state[:, col]) - 1
                    # Check the diagonals for other queens
                    for i in range(1, 8):
                        if row + i < 8 and col + i < 8:
                            temp_score += board.state[row + i, col + i]
                        if row - i >= 0 and col - i >= 0:
                            temp_score += board.state[row - i, col - i]
                        if row + i < 8 and col - i >= 0:
                            temp_score += board.state[row + i, col - i]
                        if row - i >= 0 and col + i < 8:
                            temp_score += board.state[row - i, col + i]

                    if temp_score == 0: # If the queen is not under attack at all
                        score += 1 # Increment the score
        self.h_score = score
        return score

    def get_children(self):
        """
        Get all the children of the current state.
        """
        children = []
        if self.heuristic_function == 1:
            best_child_score = np.Inf
        elif self.heuristic_function == 2:
            best_child_score = -1
            
        for row in range(self.n):
            for col in range(self.n):
                if self.state[row, col] == 1:  # If there is a queen in this position
                    for i in range(
                        self.n
                    ):  # Check all the possible positions for the queen
                        if i != col:
                            child = Board(self.n, self.heuristic_function)
                            child.state = self.state.copy()
                            child.state[
                                row, col
                            ] = 0  # Remove the queen from the current position
                            child.state[
                                row, i
                            ] = 1  # Move the queen to the new position
                            score = (
                                child.heuristic()
                            )  # Compute the heuristic score of the child

                            if (
                                self.heuristic_function == 1
                            ):  # If heuristic function 1 is used
                                if (
                                    score <= best_child_score
                                ):  # If the child is better than the best child so far:
                                    best_child_score = score
                                    children.insert(
                                        0, child
                                    )  # Insert the child at the beginning of the list
                                else:  # If the child is not better than the best child so far:
                                    children.append(
                                        child
                                    )  # Append the child to the end of the list

                            elif (
                                self.heuristic_function == 2
                            ):  # If heuristic function 2 is used
                                if (
                                    score >= best_child_score
                                ):  # If the child is better than the best child so far:
                                    best_child_score = score
                                    children.insert(
                                        0, child
                                    )  # Insert the child at the beginning of the list
                                else:  # If the child is not better than the best child so far:
                                    children.append(
                                        child
                                    )  # Append the child to the end of the list

        return children

    def search(self, time_out=-1):
        """
        Search for the solution using the hill climbing algorithm.
        """
        current_state = self
        previous_state = self

        children = current_state.get_children()  # Get the children of the initial state

        start = time.time()
        while (
            current_state.h_score != self.goal
        ):  # While the current state is not the goal state
            print(current_state.state)
            print(f'Heuristic Score = {current_state.h_score}')
            print('\n------------------------------------------------------------\n')

            if current_state != previous_state:
                # Get the children only if the current state is
                # different from the previous state to avoid getting
                # the same children again. (For efficiency)
                children = current_state.get_children()
                previous_state = current_state

            current_state = children[0]  # Get the best child

            if time_out > 0:  # If a time out is specified
                if (
                    time.time() - start > time_out
                ):  # If the algorithm takes more than #time_out seconds
                    print("Time out, solution not found.")
                    return False  # Return False: solution not found

        # If the algorithm finds the solution
        print(f"Solution for {self.n}x{self.n} Queens Problem found!")
        print(current_state.state)  # Print the solution
        print(f"Heuristic Score = {current_state.h_score}")
        print("\n------------------------------------------------------------\n")
        return True

    def search_randomness(self, time_out=-1):
        """
        Search for the solution using the hill climbing algorithm with randomness added.
        The randomess helps the algorithm to scape from the local optimum when stuck.
        The randomness is added by using a random child from the children list instead of the first.
        The randomness is applied when we do not see any improvement in the heuristic score.
        """
        current_state = self
        previous_state = self

        children = current_state.get_children()  # Get the children of the initial state

        start = time.time()
        while (
            current_state.h_score != self.goal
        ):  # While the current state is not the goal state
            print(current_state.state)
            print(f'Heuristic Score = {current_state.h_score}')
            print('\n------------------------------------------------------------\n')

            if current_state != previous_state:
                # Get the children only if the current state is
                # different from the previous state to avoid getting
                # the same children again. (For efficiency)
                children = current_state.get_children()
                previous_state = current_state

            if self.heuristic_function == 1:  # If heuristic function 1 is used
                if (
                    current_state.h_score <= children[0].h_score
                ):  # If the the best child is not better than the current state
                    current_state = children[
                        random.randint(0, len(children) - 1)
                    ]  # Use a random child
                else:  # If the best child is better than the current state
                    current_state = children[0]  # Use the best child

            elif self.heuristic_function == 2:  # If heuristic function 2 is used
                if (
                    current_state.h_score >= children[0].h_score
                ):  # If the the best child is not better than the current state
                    current_state = children[
                        random.randint(0, len(children) - 1)
                    ]  # Use a random child
                else:  # If the best child is better than the current state
                    current_state = children[0]  # Use the best child

            if time_out > 0:  # If a time out is specified
                if (
                    time.time() - start > time_out
                ):  # If the algorithm takes more than #time_out seconds
                    print("Time out, solution not found.")
                    return False  # Return False: solution not found

        # If the algorithm finds the solution
        print(f"Solution for {self.n}x{self.n} Queens Problem found!")
        print(current_state.state)  # Print the solution
        print(f"Heuristic Score = {current_state.h_score}")
        print("\n------------------------------------------------------------\n")
        return True

## Simple search with pair of queens under attack heuristic (heuristic1):

In [8]:
print('Simple search:')
start = time.time()
for i in range(8, 17):
    print(f'Running for {i}x{i} board')
    result = False
    while result == False:
        init_board = Board(i)
        result = init_board.search(time_out=1)
print(f'Total time taken: {time.time() - start} seconds.')

Simple search:
Running for 8x8 board
Solution for 8x8 Queens Problem found!
[[0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0]]
Heuristic Score = 0

------------------------------------------------------------

Running for 9x9 board
Solution for 9x9 Queens Problem found!
[[0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]]
Heuristic Score = 0

------------------------------------------------------------

Running for 10x10 board
Solution for 10x10 Queens Problem found!
[[0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]]
Heuristic Score = 0

-------------------------------------

## Search with randomness with pair of queens under attack heuristic (heuristic1):

In [10]:
print('Search with randomness:')
start = time.time()
for i in range(8, 17):
    print(f'Running for {i}x{i} board')
    result = False
    while result == False:
        init_board = Board(i)
        result = init_board.search_randomness(time_out=5)
print(f'Total time taken: {time.time() - start} seconds.')

Search with randomness:
Running for 8x8 board
Solution for 8x8 Queens Problem found!
[[0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0]]
Heuristic Score = 0

------------------------------------------------------------

Running for 9x9 board
Solution for 9x9 Queens Problem found!
[[0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0]]
Heuristic Score = 0

------------------------------------------------------------

Running for 10x10 board
Solution for 10x10 Queens Problem found!
[[0 0 0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]]
Heuristic Score = 0

----------------------------

## A lot of times, simple search won't converge to an answer:

In [31]:
for i in range(10):
    init_board = Board(16)
    init_board.search(time_out=2)

Time out, solution not found.
Time out, solution not found.
Time out, solution not found.
Time out, solution not found.
Time out, solution not found.
Time out, solution not found.
Time out, solution not found.
Time out, solution not found.
Time out, solution not found.
Time out, solution not found.


## But when it does, it's pretty fast:

In [35]:
start = time.time()
init_board = Board(16)
init_board.search(time_out=2)
print(f'Total time taken: {time.time() - start} seconds.')

Solution for 16x16 Queens Problem found!
[[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]]
Heuristic Score = 0

------------------------------------------------------------

Total time taken: 1.28664231300354 seconds.


## Adding randomness helps it find the right answer almost all the time, but takes more time:

In [43]:
start = time.time()
init_board = Board(16)
init_board.search_randomness()
print(f'Total time taken: {time.time() - start} seconds.')

Solution for 16x16 Queens Problem found!
[[0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]]
Heuristic Score = 0

------------------------------------------------------------

Total time taken: 22.96498465538025 seconds.


## Tests above were using a heuristic function that was summing the number of queens attacking each queen.
---
# Now we will use another heuristic function:
- ### "The number of queens that are not under attack."


The goal heuristic score here will be N for a NxN chess board.

In [82]:
init_board = Board(8, heuristic_function=2)
res = init_board.search(time_out=5)

[[0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]]
Heuristic Score = 3

------------------------------------------------------------

[[0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]]
Heuristic Score = 5

------------------------------------------------------------

Solution for 8x8 Queens Problem found!
[[0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0]]
Heuristic Score = 8

------------------------------------------------------------



In [91]:
init_board = Board(10, heuristic_function=2)
res = init_board.search_randomness(time_out=100)

[[0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0]]
Heuristic Score = 2

------------------------------------------------------------

[[0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0]]
Heuristic Score = 4

------------------------------------------------------------

[[0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 0]]
Heuristic Score = 5

------------------------------------------------------------

[[0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 

# As you can see above:
The regular search algorithm will get stuck in a local optimum and will not find the answer a lot of times.
While the search with randomness algorithm, almost always finds the answer but takes a lot more time to find it, compared to a regular search that converges.

# For example:
You can see above that we executed the regular search algorithm for 10 times on the 16x16 problem, and non of them converged to an answer. (It would get to an answer in less than 2 seconds. If surpasses 2 seconds, its getting stuck)

After executing for a number of more times, we got to an answer in 1.2 seconds.
Running the random search algorithm we can almost always get the answer but it costs us the time. In our example, we got the correct answer on the first execution, but in 23 seconds. A lot more than our regular search.

It's a trade off between time and accuracy.

# The Heuristic Functions Used (Summary):
##### 1. Computes and sums the number of queens attacking each queen.
##### 2. Counts the number of queens that are not under attack.

These are the heuristic functions we use to compare states with each other.

## We can visualize our search for an 8x8 chess board using the first heuristic and see the process of finding the answer:

In [61]:
init_board = Board(12)
result = init_board.search()

[[0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0]]
Heuristic Score = 9

------------------------------------------------------------

[[0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0]]
Heuristic Score = 7

------------------------------------------------------------

[[0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 